In [1]:
#!pip install seaborn
#!pip install openpyxl
#!pip install sklearn
#!pip install yfinance 

In [2]:
import pandas as pd
import numpy as np
import random as rd
import seaborn as sbs
import yfinance as yf
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.svm import SVR
from matplotlib import pyplot
import time
import csv
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error

In [3]:
path_name_results='../results/'
file_result = 'Result_SVR_IBM_stock_prices.csv'

In [4]:
# Define the stock symbol (IBM)
stock_symbol = "IBM"

# Define the start and end dates for the historical data
start_date = "2017-01-01"
end_date = "2023-09-21"

# Download historical data
dataset = yf.download(stock_symbol, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1690 entries, 2017-01-03 to 2023-09-20
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1690 non-null   float64
 1   High       1690 non-null   float64
 2   Low        1690 non-null   float64
 3   Close      1690 non-null   float64
 4   Adj Close  1690 non-null   float64
 5   Volume     1690 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 92.4 KB


In [6]:
#verifica se existe variáveis nulas 
dataset.isna().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [7]:
def salvar_resultado(nm_dataset, ds_best_param, n_time_steps, MSE, RMSE, MAE, MAPE, sMAPE, Duration):
  #Script to write training cycle results
  data = [nm_dataset, ds_best_param, n_time_steps, MSE, RMSE, MAE, MAPE, sMAPE, Duration]
  fields = ['Dataset','Best Params','n_time_steps','MSE','RMSE','MAE','MAPE','sMAPE','Duration']
  with open(f'{path_name_results}{file_result}', "a",newline='') as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerow(data)  
  print(fields)
  print(data)
    
#Script to create the results file
def criar_arquivo_resultado():
  fields = ['Dataset','Best Params','n_time_steps','MSE','RMSE','MAE','MAPE','sMAPE','Duration']
  with open(f'{path_name_results}{file_result}', "w",newline='') as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerow(fields)    

In [8]:
def previsao_MLP(nm_dataset, dataset, n_time_steps):

    # dataframe tratament
  df = pd.DataFrame()
  df['Open']=dataset['Open']  

  # time serire transform - shit 1 step time

  for n_step in range(1,n_time_steps+1,1):
    df['vl-'+str(n_step)]=dataset['Open'].shift(n_step)  
      
  df.dropna(inplace=True)
  
  
  
  #Split dataset in treinam /  80% treinam  20% test
  nlinhas = int(np.round(df.shape[0] *0.80)) # 
  
  max_size_train_split = int(np.round(nlinhas / 5)) 
  max_size__test_split = int(np.round((df.shape[0] - nlinhas) / 5))
  size_split = 5
  X_train = df.iloc[0:nlinhas,1: 1 + n_time_steps]
  y_train = df.iloc[0:nlinhas,0].values

  X_test = df.iloc[nlinhas:dataset.shape[0],1: 1 + n_time_steps] 
  y_test = df.iloc[nlinhas:dataset.shape[0],0].values
  
  
  # Stores the training execution start time
  Hora_Inicio = time.time()
  
  #Create SVR Regressor
  modelo  = SVR(kernel='rbf')
  
  # Validados cruzado para séries temporais para 5 conjuntos de dados
  ts_cv = TimeSeriesSplit(
      n_splits=size_split, # Number of divisions
      max_train_size=max_size_train_split,    # maximum size of each set. of training
      gap=2, # number of samples to exclude between each training and testing set
      test_size=max_size__test_split, # maximum size of each set. of test.
  )
  C = [ 12550, 125550, 1255555] # Parâmetro de regularização
  gamma = [ 0.00001, 0.000001, 0.0000001, 0.00000001] # Coeficiente da função kernel  
  epsilon = [0.1, 0.01, 0.001, 0.0001]

  hyper_params = [{'kernel': ['rbf'],'C': C, 'gamma':gamma, 'epsilon':epsilon}]  

  modelo = SVR(max_iter=500)
  grid = GridSearchCV(modelo,param_grid=hyper_params,verbose=20,n_jobs=-4,cv=ts_cv, scoring='neg_mean_absolute_percentage_error')
  grid.fit(np.array(X_train), np.array(y_train))
  
  resultado = str(grid.best_params_)
  

  predict=grid.predict(np.array(X_test))   

  # Stores the training execution end time
  Hora_Fim = time.time()   

  #Calculate the duration of the training execution
  Duracao = Hora_Fim - Hora_Inicio   
  
  #Mean Squared Error (Mean Squared Difference Between Estimated Values and Actual Values) - MSE
  MSE = mean_squared_error(y_test, predict)   

  # Square Root of Mean Error - RMSE
  RMSE = np.sqrt(mean_squared_error(y_test, predict))   

  # Mean Absolute Distance or Mean Absolute Error - MAE
  MAE= median_absolute_error(y_pred=predict, y_true = y_test)  

  #Calculate the MAPE (Mean Absolute Percentage Error)
  MAPE = ((np.mean(np.abs(y_test -predict) / (y_test)))) * 100    

  sMAPE = round(
		np.mean(
			np.abs(predict - y_test) /
			((np.abs(predict) + np.abs(y_test)))
		)*100, 2
	)

  salvar_resultado(nm_dataset, resultado, n_time_steps, MSE, RMSE, MAE, MAPE, sMAPE, Duracao)



In [9]:
#create file to results
criar_arquivo_resultado()

print('forecast for new sunspot number')
for n_time_steps in range(1,13): #predict with 1 to 12 past values of medition
    grid = previsao_MLP('IBM', dataset, n_time_steps)

forecast for new sunspot number
Fitting 5 folds for each of 48 candidates, totalling 240 fits


c:\Python\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['IBM', "{'C': 12550, 'epsilon': 0.1, 'gamma': 1e-05, 'kernel': 'rbf'}", 1, 49.94503440358657, 7.067180088520921, 5.3042621070613905, 4.441026602064264, 2.15, 11.146066427230835]
Fitting 5 folds for each of 48 candidates, totalling 240 fits


c:\Python\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['IBM', "{'C': 12550, 'epsilon': 0.1, 'gamma': 1e-05, 'kernel': 'rbf'}", 2, 227.60749679187165, 15.086666192100614, 15.189831569530725, 11.117801290456528, 5.9, 6.15429425239563]
Fitting 5 folds for each of 48 candidates, totalling 240 fits


...

['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['IBM', "{'C': 12550, 'epsilon': 0.0001, 'gamma': 1e-06, 'kernel': 'rbf'}", 10, 16.202100950141855, 4.025183343667945, 3.709675002190579, 2.6151364061578546, 1.33, 7.818742036819458]
Fitting 5 folds for each of 48 candidates, totalling 240 fits


c:\Python\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['IBM', "{'C': 125550, 'epsilon': 0.0001, 'gamma': 1e-07, 'kernel': 'rbf'}", 11, 12.54560695744536, 3.541977831303488, 2.7881143574704765, 2.13300615268845, 1.08, 4.717573642730713]
Fitting 5 folds for each of 48 candidates, totalling 240 fits


c:\Python\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['IBM', "{'C': 125550, 'epsilon': 0.1, 'gamma': 1e-07, 'kernel': 'rbf'}", 12, 11.066817747408757, 3.3266826941277037, 2.561349558767688, 2.004356989626636, 1.01, 5.169879913330078]
